In [ ]:
import os
import Pandas
import numpy as np
import pyvisa
import time
import datetime
from matplotlib import pyplot as plt

In [ ]:
#--- Record date and time ---
dt_now = datetime.datetime.now()
today = dt_now.strftime("%Y%m%d-%H:%M:%S")

In [ ]:
#--- Create Output Directory ---
dir_path = ''
dir_name = today + '-testB5-NamPham'
dir_path_name = os.path.join(dir_path, dir_name)
os.mkdir(dir_path_name, exist_ok=True)

In [ ]:
#--- Obtaining the VISA addresses of connected devices ---
rm = pyvisa.ResourceManager()
print(f"VISA addresses of connected devices are as below:\n{rm.list_resources()}")

In [ ]:
#--- Defining device name (COPY VISA ADDRESSES & EDIT) ---
scope = rm.open_resource(, timeout=10000)
afg = rm.open_resource(, timeout=1000)

In [ ]:
#--- Confirm oscilloscope ID ---
scope_id = scope.query("*IDN?")
afg_id = afg.query("*IDN?")
print(f"scope ID = {scope_id}")
print(f"function generator ID = {afg_id}")

In [ ]:
#--- Asking for Input ---
freq = float(input("Frequency (MHz):")) # frequency

while True:                             # pulse width
    pulse_width = int(input("Pulse Width (%):"))
    if pulse_width < 10 or pulse_width > 90:
        print("Width must be between 10 percent and 90 percent")
    else:
        break

while True:                             # amplitude
    amp = float(input("Amplitude (VPP):"))
    if amp < 0.8 or amp > 1.3:
        print("Amplitude not in effective range (0.8V to 1.3V)")
    else:
        break

offset = float(input("Offset (V):"))    # offset

while True:                             # phase
    phase = float(input("Offset (V):"))
    if phase < -180 or phase > 180:
        print("Phase must be bewteen -180 and 180")
    else:
        break

In [ ]:
#--- Function Generator Output Setup ---
afg.write("FUNC:SHAP PULS")
afg.write(f"FREQ {freq}E6")
afg.write(f"PULS:{pulse_width}")
afg.write(f"VOLT {amp}")
afg.write(f"VOLT:OFFS {offset}")
afg.write(f"PHAS {phase}")
afg.write("OUTP:IMP 50")

In [ ]:
#--- Oscilloscope Setup ---
scope.write(":WAV:SOUR CHAN1")
scope.write(":WAV:MODE NORM")
scope.write(":WAV:FORM ASCii")
scope.write(f":TIMEBASE:SCALE {5e-6/freq}")
scope.write(":TRIG:MODE EDGE")
scope.write(":TRIG:EDGE:SOUR CHAN1")
scope.write(":TRIG:EDGE:SLOPE POSITIVE")
scope.write(f":TRIG:LEVEL {offset}")
scope_dt = float(scope.query(":WAV:XINC?"))
scope_dd = float(scope.query(":WAV:YINC?"))
scope_ot = float(scope.query(":WAV:XOR?"))
scope_points = int(scope.query(":WAV:POINTs?"))

In [ ]:
#--- Measurement and Saving ---
afg.write("OUTP ON")
data = scope.query(":WAV:DATA?")
time.sleep(1.0)
afg.write("OUTP OFF")

voltage = np.array([float(v) for v in data.split(',')])
time = scope_dt * np.arange(scope_points)

In [ ]:
#--- Saving raw data ---
np.savetxt(f"{dir_path_name}/waveform_data.csv", np.column_stack((time, voltage)),
           delimiter=",", header="Time(s), Voltage(V)", comments='')

In [ ]:
#--- Measurement and Saving ---
afg.write("OUTP ON")
data = scope.query(":WAV:DATA?")
time.sleep(1.0)
afg.write("OUTP OFF")

In [ ]:
#--- Visualization ---
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(111)

ax.plot(time/1e6 ,voltage, drawstyle='steps-mid')
ax.set_xlabel("Time (μs)")
ax.set_ylabel("Voltage (V)")
ax.set_title("Waveform")
ax.grid(True)

plt.savefig(f"{dir_path_name}/waveform_plt.png")
plt.close()

In [ ]:
#--- Saving setup parameters ---
metadata = {
    "Function Generator": {
        "ID": afg_id,
        "Waveform": "Pulse",
        "Frequency": f"{freq} MHz",
        "Amplitude": f"{amp} Vpp",
        "Offset": f"{offset} V",
        "Duty": f"{pulse_width}%",
        "Phase": f"{phase} deg",
        "Impedance": "50 Ω"
    },
    "Oscilloscope": {
        "ID": scope_id,
        "Timebase Scale": f"{5000/freq} ns/div",
        "Time Resolution (XINC)": f"{scope_dt} s",
        "Time Offset (XOR)": f"{scope_ot} s",
        "Points": scope_points
    }
}
with open(f"{dir_path_name}/metadata.txt", "w") as f:
    for section, entries in metadata.items():
        f.write(f"[{section}]\n")
        for key, val in entries.items():
            f.write(f"{key}: {val}\n")
        f.write("\n")